# sqlite-vec 入门指南

本指南将引导您完成 `sqlite-vec` 的基本使用，包括安装、加载、处理向量以及结合本地句子转换器模型。

## 1. 安装 sqlite-vec

首先，您需要安装 `sqlite-vec` PyPi 包。可以使用 pip 进行安装：

In [2]:
%pip install sqlite-vec sentence-transformers --quiet

Note: you may need to restart the kernel to use updated packages.


## 2. 加载 sqlite-vec

安装完成后，使用 `sqlite_vec.load()` 函数将 `sqlite-vec` SQL 函数加载到 SQLite 连接中。

In [3]:
import sqlite3
import sqlite_vec

# 创建一个内存数据库或连接到现有数据库文件
db = sqlite3.connect(':memory:') # 或者 db = sqlite3.connect('my_vectors.db')

# 启用扩展加载
db.enable_load_extension(True)

# 加载 sqlite-vec 扩展
sqlite_vec.load(db)

# （可选）加载后禁用扩展加载，以增强安全性
db.enable_load_extension(False)

# 验证 sqlite-vec 版本
vec_version, = db.execute("select vec_version()").fetchone()
print(f"sqlite-vec version: {vec_version}")

sqlite-vec version: v0.1.6


## 3. 处理向量

`sqlite-vec` 支持多种方式处理向量数据。

### 3.1 使用 Python 列表

如果您的向量是 Python 中的浮点数列表，可以使用 `sqlite_vec.serialize_float32()` 将其转换为 `sqlite-vec` 使用的紧凑 BLOB 格式。

In [5]:
from sqlite_vec import serialize_float32

embedding_list = [0.1, 0.2, 0.3, 0.4]
serialized_embedding = serialize_float32(embedding_list)
print(f"Serialized vector: {serialized_embedding}")
# 打印数据类型
print(f"Serialized vector type: {type(serialized_embedding)}")
result = db.execute('select vec_length(?)', [serialized_embedding])
length = result.fetchone()[0]
print(f"Length of vector from list: {length}")

Serialized vector: b'\xcd\xcc\xcc=\xcd\xccL>\x9a\x99\x99>\xcd\xcc\xcc>'
Serialized vector type: <class 'bytes'>
Length of vector from list: 4


### 3.2 使用 NumPy 数组

如果您的向量是 NumPy 数组，Python SQLite 包允许您直接传递。确保将数组元素转换为 32 位浮点数 (`np.float32`)。

In [7]:
import numpy as np

embedding_numpy = np.array([0.1, 0.2, 0.3, 0.4, 0.5])
embedding_numpy_f32 = embedding_numpy.astype(np.float32)
print(f"NumPy array: {embedding_numpy_f32}")
# 打印数据类型
print(f"NumPy array type: {type(embedding_numpy_f32)}")
result = db.execute('SELECT vec_length(?)', [embedding_numpy_f32])
length = result.fetchone()[0]
print(f"Length of vector from NumPy array: {length}")

NumPy array: [0.1 0.2 0.3 0.4 0.5]
NumPy array type: <class 'numpy.ndarray'>
Length of vector from NumPy array: 5


## 4. 结合本地 Sentence Transformer 模型

我们可以使用 `sentence-transformers` 库加载本地模型生成文本嵌入，并将其存储到 `sqlite-vec` 中。

In [8]:
from sentence_transformers import SentenceTransformer

model_path = r'C:\Users\k\Desktop\BaiduSyncdisk\baidu_sync_documents\hf_models\bge-m3'

try:
    model = SentenceTransformer(model_path)
    embedding_dim = model.get_sentence_embedding_dimension()
    print(f"Loaded model from: {model_path}")
    print(f"Embedding dimension: {embedding_dim}")
except Exception as e:
    print(f"Error loading model: {e}")
    model = None
    embedding_dim = None

c:\Users\k\.conda\envs\SmartImageFinder\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loaded model from: C:\Users\k\Desktop\BaiduSyncdisk\baidu_sync_documents\hf_models\bge-m3
Embedding dimension: 1024


现在，让我们生成一个文本嵌入，并将其存储到数据库中。

In [9]:
if model:
    sample_text = "这是一个示例文本，用于生成嵌入。"
    text_embedding = model.encode(sample_text).astype(np.float32)
    
    print(f"Generated embedding for: '{sample_text}'")
    print(f"Embedding shape: {text_embedding.shape}")
    
    # 创建一个表来存储文本和它们的嵌入
    # 说明: CREATE TABLE IF NOT EXISTS - 如果表不存在则创建表
    # id INTEGER PRIMARY KEY - 自增长的主键字段
    # text TEXT - 存储原始文本内容
    # embedding BLOB - 使用二进制大对象类型存储向量，大小为模型输出维度
    db.execute(f"CREATE TABLE IF NOT EXISTS documents (id INTEGER PRIMARY KEY, text TEXT, embedding BLOB({embedding_dim}))")
    
    # 插入数据到documents表
    # 说明: INSERT INTO - 插入新行数据
    # (text, embedding) - 指定要插入值的列名
    # VALUES (?, ?) - 使用参数化查询避免SQL注入风险
    db.execute("INSERT INTO documents (text, embedding) VALUES (?, ?)", (sample_text, text_embedding))
    db.commit()
    print("Text and embedding stored in the database.")
    
    # 检索并验证存储的数据
    # 说明: SELECT - 查询表中的数据
    # vec_length(embedding) - 计算向量长度的sqlite-vec函数
    # vec_to_json(embedding) - 将二进制向量转换为JSON格式的sqlite-vec函数
    # WHERE id = 1 - 筛选条件，获取ID为1的记录
    retrieved_row = db.execute("SELECT text, vec_length(embedding), vec_to_json(embedding) FROM documents WHERE id = 1").fetchone()
    if retrieved_row:
        retrieved_text, retrieved_length, retrieved_embedding_json = retrieved_row
        print(f"\nRetrieved text: {retrieved_text}")
        print(f"Retrieved embedding length: {retrieved_length}")
        # print(f"Retrieved embedding (first 5 dims): {retrieved_embedding_json[:50]}...") # 打印部分嵌入以验证
else:
    print("Sentence Transformer model not loaded. Skipping embedding generation and storage.")

Generated embedding for: '这是一个示例文本，用于生成嵌入。'
Embedding shape: (1024,)
Text and embedding stored in the database.

Retrieved text: 这是一个示例文本，用于生成嵌入。
Retrieved embedding length: 1024


### 4.1 官方基础示例



In [21]:
# 官方 `vec0` 基础示例代码 - 演示了 vec0 模块的基本用法

# 注意: 虽然我们在这个例子中创建一个新的数据库连接，但实际应用中可以使用一个连接管理所有表
# 这里创建一个新连接是为了演示一个完整、独立的示例
import sqlite3
import sqlite_vec
import numpy as np
from sqlite_vec import serialize_float32

# 创建一个新的内存数据库并加载 sqlite-vec 扩展
db_vec0 = sqlite3.connect(":memory:")
db_vec0.enable_load_extension(True)
sqlite_vec.load(db_vec0)
db_vec0.enable_load_extension(False)

# 检查 SQLite 和 sqlite-vec 版本
sqlite_version, vec_version = db_vec0.execute("SELECT sqlite_version(), vec_version()").fetchone()
print(f"SQLite 版本: {sqlite_version}, sqlite-vec 版本: {vec_version}")

# 准备示例数据 - 每项包含 ID 和 4 维向量
items = [
    (1, [0.1, 0.1, 0.1, 0.1]),  # ID 1 的向量
    (2, [0.2, 0.2, 0.2, 0.2]),  # ID 2 的向量
    (3, [0.3, 0.3, 0.3, 0.3]),  # ID 3 的向量，与查询向量完全匹配
    (4, [0.4, 0.4, 0.4, 0.4]),  # ID 4 的向量
    (5, [0.5, 0.5, 0.5, 0.5]),  # ID 5 的向量
]

# 查询向量 - 我们将寻找与此向量最相似的向量
query = [0.3, 0.3, 0.3, 0.3]

# 创建一个虚拟表用于向量搜索
# vec0 是 sqlite-vec 提供的向量索引类型，适用于小型数据集的精确搜索
# embedding 是存储向量的列名，float[4]表示4维浮点向量
db_vec0.execute("CREATE VIRTUAL TABLE vec_items USING vec0(embedding FLOAT[4])")

# 向虚拟表中插入示例数据
with db_vec0:
    for item in items:
        db_vec0.execute(
            "INSERT INTO vec_items(rowid, embedding) VALUES (?, ?)",
            [item[0], serialize_float32(item[1])]
        )

# 执行向量相似性搜索
# embedding MATCH ? - 使用MATCH操作符执行向量相似性搜索
# ORDER BY distance - 按相似度距离排序（距离越小越相似）
# LIMIT 3 - 只返回前3个最相似的结果
rows = db_vec0.execute(
    """
    SELECT
      rowid,
      distance
    FROM vec_items
    WHERE embedding MATCH ?
    ORDER BY distance
    LIMIT 3
    """,
    [serialize_float32(query)]
).fetchall()

# 打印搜索结果
print("最相似向量的搜索结果 (ID, 距离):")
for row in rows:
    print(f"ID: {row[0]}, 距离: {row[1]}")

SQLite 版本: 3.45.3, sqlite-vec 版本: v0.1.6
最相似向量的搜索结果 (ID, 距离):
ID: 3, 距离: 0.0
ID: 4, 距离: 0.19999998807907104
ID: 2, 距离: 0.20000001788139343


## 5. 使用最新版本的 SQLite

`sqlite-vec` 的某些功能需要最新版本的 SQLite 库。您可以通过 `sqlite3.sqlite_version` 查看 Python 环境中使用的 SQLite 版本。

目前，**推荐使用 SQLite 3.41 或更高版本**。

In [10]:
print(f"Python's SQLite version: {sqlite3.sqlite_version}")

Python's SQLite version: 3.45.3


有几种方法可以升级 Python 使用的 SQLite 版本：
1.  **自行编译 SQLite 版本**：编译最新 SQLite 并使用环境变量（如 `LD_PRELOAD`）强制 Python 使用它。
2.  **使用 `pysqlite3`**：这是一个第三方 PyPi 包，捆绑了最新版本的 SQLite。
3.  **升级 Python 版本**：较新的 Python 版本通常会附带较新的 SQLite 版本。

## 6. MacOS 默认阻止 SQLite 扩展

MacOS 自带的默认 SQLite 库不支持 SQLite 扩展，这可能导致 `AttributeError: 'sqlite3.Connection' object has no attribute 'enable_load_extension'`。

解决方法包括：
*   使用 Homebrew 版本的 Python (`brew install python`)。
*   参考“使用最新版本的 SQLite”部分中提到的其他方法。

## 7. 清理

关闭数据库连接。

In [ ]:
db.close()